### Agents
Agents will provide a way for us to add additional sources of information that will build on our RAG approach.  We may need to execute code or search different information sources to build a complex context to answer user queries.

### Tools

In [ ]:
# %pip install langchain_community -q
# %pip install tavily-python -q
# %pip install python-dotenv -q
# %pip install langchainhub -q
%pip install wikibase-rest-api-client mediawikiapi -q
%pip install yfinance

In [ ]:
import os
import boto3

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsInput
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import AIMessage, HumanMessage
from langchain import hub
from dotenv import load_dotenv

load_dotenv()
# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

### Agent Setup
We need three components to make up our agent
1) Tools we plan to use to execute respond to input
2) LLM we plan to use as our logic executor
3) Agent type we plan to use for this use case

#### Tavily Search
Tavily's Search API is a search engine built specifically for AI agents (LLMs) that LangChain supports as a library import.

*Note: This tool requires an API Key that will be loaded in your .env file as follows*.  

>TAVILY_API_KEY=tvly-XXXXXXXXXXXXXXXXXXX

To exectute this notebook [visit](https://app.tavily.com/) for an API key.


In [ ]:
test = os.getenv('TAVILY_API_KEY')
print("API=",test)

In [ ]:
# 1 Start with a simple web search tool:
tools = [TavilySearchResults(max_results=1, TAVILY_API_KEY = os.getenv('TAVILY_API_KEY'))]

# 2 LLM, Let's use Claude's Haiku
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# Grab our LLM of choice
model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 0.9,
    "stop_sequences": ["\n\nHuman"],
}
llm = BedrockChat(
    client=aws_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# Get the prompt to use with our agent
# The hub provides sample templates for each agent type
prompt = hub.pull("hwchase17/structured-chat-agent")


#print("Tools:", tools)
#print("Prompt", prompt)



#### Our logic prompt
Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```

{{

  "action": $TOOL_NAME,

  "action_input": $INPUT

}}

```

Follow this format:

Question: input question to answer

Thought: consider previous and subsequent steps

Action:

```

$JSON_BLOB

```

Observation: action result

... (repeat Thought/Action/Observation N times)

Thought: I know what to respond

Action:

```

{{

  "action": "Final Answer",

  "action_input": "Final response to human"

}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation

In [ ]:
# 3 Construct the create_structured_chat_agent agent
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
agent = create_structured_chat_agent(llm, tools, prompt)

# run the agent
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

agent_executor.invoke({"input": "What happened to the economy historically when tariffs were added to US imports?"})

In [ ]:
response = agent_executor.invoke({"input": "Hi my name is Darren"})
last_input = response['input']
last_output = response['output']

#### Can agents keep track of our conversation and use the context?
If we save the results, LangChain supports chat history

In [ ]:
response = agent_executor.invoke({"input": "What's my name?"})

In [ ]:
response = agent_executor.invoke({"input": "What's my name?", 
                                  "chat_history": [
                                        HumanMessage(content=last_input),
                                        AIMessage(content=last_output),
                                                  ],})


#### Wiki Data Tool
Let's add anther tool and observe how our Agent can use both tools to answer input questions

In [ ]:
# New tool WikiData
wikidata = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())
# Add WikiData and Search Results
tools = [wikidata, TavilySearchResults(max_results=1, TAVILY_API_KEY = os.getenv('TAVILY_API_KEY'))]
agent = create_structured_chat_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)
agent_executor.invoke({"input": "Who was the historical figure named Richard Feynman?"})

Now let's review our original question and see how the agent responds to the query.

In [ ]:
agent_executor.invoke({"input": "What happened to the economy historically when tariffs were added to US imports?"})

#### Homework
Use the [langchain tool](https://python.langchain.com/api_reference/community/tools.html) list to find a useful tool other than Wikidata or Tavily to explore what it can do 

In [ ]:
# Your tool here
# 1 Start with a simple web search tool:
#tools = -> FIND A GOOD TOOL


# 2 LLM, Let's use Claude's Haiku
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# Grab our LLM of choice
model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 0.9,
    "stop_sequences": ["\n\nHuman"],
}
llm = BedrockChat(
    client=aws_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# Get the prompt to use with our agent
# The hub provides sample templates for each agent type
prompt = hub.pull("hwchase17/structured-chat-agent")

# 3 Construct the create_structured_chat_agent agent
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
agent = create_structured_chat_agent(llm, tools, prompt)

# run the agent
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

agent_executor.invoke({"input": "What happened with the economy today?"})